<a href="https://colab.research.google.com/github/jenboland/DailyKnowledge/blob/master/Meridian_MMM_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings

import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc as pm
import seaborn as sns

warnings.filterwarnings("ignore", category=FutureWarning)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Install meridian: from PyPI @ latest release
!pip install --upgrade google-meridian[colab]

# Install meridian: from PyPI @ specific version
#!pip install google-meridian==1.0.0

# Install meridian: from GitHub @HEAD
#!pip install --upgrade "google-meridian[colab] @ git+https://github.com/google/meridian.git"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 813.6/813.6 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.7/590.7 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 84.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.0/189.0 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 84.4 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.13.1
    Uninstalling scipy-1.13.1:
      Successfully uninstalled scipy-1.13.1
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully unin

In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp
import arviz as az

import IPython

from meridian import constants
from meridian.data import load
from meridian.data import test_utils
from meridian.model import model
from meridian.model import spec
from meridian.model import prior_distribution
from meridian.analysis import optimizer
from meridian.analysis import analyzer
from meridian.analysis import visualizer
from meridian.analysis import summarizer
from meridian.analysis import formatter

# check if GPU is available
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Num CPUs Available: ", len(tf.config.experimental.list_physical_devices('CPU')))

Your runtime has 13.6 gigabytes of available RAM

Num GPUs Available:  0
Num CPUs Available:  1


In [5]:
path_ = "/content/drive/MyDrive/Colab Notebooks/"

In [6]:
bing = pd.read_csv(path_+'cafb_bing_daily.csv', parse_dates=['Date'], index_col=['Date']).resample('W').sum()

#old file
#dm = pd.read_csv(path_+'cafb_direct_mail_daily.csv', parse_dates=['Date'], index_col=['Date']).resample('W').sum()
#new file
dm = pd.read_csv(path_+'cafb_dm_v3.csv', parse_dates=['Date'], index_col=['Date']).resample('W').sum()

email = pd.read_csv(path_+'cafb_email_daily.csv', parse_dates=['Date'], index_col=['Date']).resample('W').sum()
fb = pd.read_csv(path_+'cafb_facebook_daily.csv', parse_dates=['Date'], index_col=['Date']).resample('W').sum()
paid_search = pd.read_csv(path_+'cafb_google_paid_search.csv', parse_dates=['Date'], index_col=['Date']).resample('W').sum()
prog = pd.read_csv(path_+'cafb_programmatic_daily.csv', parse_dates=['Date'], index_col=['Date']).resample('W').sum()
#old file
#sales = pd.read_csv(path_+'cafb_transactions.csv', parse_dates=['Date'], index_col=['Date']).resample('W').sum()
sales = pd.read_csv(path_+'cafb_daily_no_monthly.csv', parse_dates=['Date'], index_col=['Date']).resample('W').sum()

ftf = pd.read_excel(path_+'CAFB MMM Telefundraising Data 2022-2024v2.xlsx', sheet_name = 'transposed', parse_dates=['timestamp']).fillna(0).rename(columns={'timestamp':'Date'}).set_index('Date').resample('W').sum()
ga4 = pd.read_csv(path_+'cafb_ga4_sessions.csv', parse_dates=['Date'], index_col=['Date']).resample('W').sum()

fb = fb.rename(columns={'Impressions':'FB_Impressions', 'Spend':'FB_Spend'})
ftf = ftf.rename(columns={'calls':'FTF_Impressions', 'funds':'FTF_Spend'})

consumer_demand = pd.read_excel(path_+'Economic_Data.xlsx', sheet_name='Consumer_Demand', index_col=None)
consumer_demand = consumer_demand.set_index('Date')
consumer_demand.head()


,Consumer_Demand
Date,
2022-01-02,0
2022-01-09,28
2022-01-16,24
2022-01-23,0
2022-01-30,24


In [7]:
df = pd.concat([bing, dm, email, fb, paid_search, prog, ftf ,ga4, sales], axis=1).fillna(0).head(144).reset_index()
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')

df = pd.merge(df, consumer_demand, on='Date')

In [8]:
# prompt: I have a column consumer demand. I want to divide Bing_Branded_Impressions, GoogleAds_Branded_Impressions by Consumer_Demand. If Consumer_Demand is 0, then divide by 1

# Assuming your dataframe is named 'df'
#df['Bing_Branded_Impressions_Normalized'] = df['Bing_Branded_Impressions'] / df['Consumer_Demand'].replace(0, 1)
#df['GoogleAds_Branded_Impressions_Normalized'] = df['GoogleAds_Branded_Impressions'] / df['Consumer_Demand'].replace(0, 1)

In [13]:
df["eoy_campaign"] = df["Date"].apply(lambda x: 1 if x.month in [11, 12] else 0)
df["end_of_year"] = ((df["Date"].dt.month == 12) & (df["Date"].dt.day == 31)) | ((df["Date"].dt.month == 1) & (df["Date"].dt.day == 1))
df["end_of_year"] = df["end_of_year"].astype(int)
df['trend'] = df['Consumer_Demand']

In [14]:
df.head(2)

,Date,Bing_Branded_Impressions,Bing_Branded_Spend,Bing_NonBranded_Impressions,Bing_NonBranded_Spend,DM_Spend,DM_Impressions,Email_Impressions,Email_Spend,FB_Spend,...,Sessions,Revenue,Consumer_Demand,eoy_campaign,end_of_year,GoogleAds_Impressions,GoogleAds_Spend,Bing_Impressions,Bing_Spend,trend
0,2022-01-02,1.0,3.26,3852.0,210.97,0.0,0.0,0.0,0.0,0.0,...,1803.0,36167.35,0,0,0,1512.0,283.49,3853.0,214.23,0
1,2022-01-09,64.0,8.25,4482.0,757.49,0.0,0.0,0.0,0.0,0.0,...,6907.0,72692.65,28,0,0,7891.0,1797.22,4546.0,765.74,28


In [15]:
df['GoogleAds_Impressions'] = df['GoogleAds_Branded_Impressions'] + df['GoogleAds_NonBranded_Impressions']
df['GoogleAds_Spend'] = df['GoogleAds_Branded_Spend'] + df['GoogleAds_NonBranded_Spend']

df['Bing_Impressions'] = df['Bing_Branded_Impressions'] + df['Bing_NonBranded_Impressions']
df['Bing_Spend'] = df['Bing_Branded_Spend'] + df['Bing_NonBranded_Spend']

In [16]:
df = df[['Date', 'DM_Spend',
       'DM_Impressions', 'Email_Impressions', 'Email_Spend', 'FB_Spend',
       'FB_Impressions', 'Programatic_Spend',
       'Programatic_Impressions', 'FTF_Spend', 'FTF_Impressions',  'Revenue', 'eoy_campaign', 'trend',
       'GoogleAds_Impressions', 'Bing_Impressions', 'GoogleAds_Spend',
       'Bing_Spend','Sessions']]

In [17]:
df.columns

Index(['Date', 'DM_Spend', 'DM_Impressions', 'Email_Impressions',
       'Email_Spend', 'FB_Spend', 'FB_Impressions', 'Programatic_Spend',
       'Programatic_Impressions', 'FTF_Spend', 'FTF_Impressions', 'Revenue',
       'eoy_campaign', 'trend', 'GoogleAds_Impressions', 'Bing_Impressions',
       'GoogleAds_Spend', 'Bing_Spend', 'Sessions'],
      dtype='object')

In [18]:
df['geo'] = 'US'

<ipython-input-18-aceeeb5b4cad>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['geo'] = 'US'


In [19]:
coord_to_columns = load.CoordToColumns(
    time='Date',
    geo='geo',
    controls=['eoy_campaign', 'trend'],
    kpi='Revenue',
    #revenue_per_kpi='revenue_per_kpi',
    media=[
        'GoogleAds_Impressions',
        'Bing_Impressions',
        'DM_Impressions',
        'Email_Impressions',
        'FB_Impressions',
        'Programatic_Impressions',
        'FTF_Impressions'
    ],
    media_spend=[
        'GoogleAds_Spend',
        'Bing_Spend',
        'DM_Spend',
        'Email_Spend',
        'FB_Spend',
        'Programatic_Spend',
        'FTF_Spend'
    ],
    #organic_media=['Sessions'],
    #non_media_treatments=['Promo'],
)

In [20]:
correct_media_to_channel = {
    'GoogleAds_Impressions': 'GoogleAds',
    'Bing_Impressions': 'Bing',
    'DM_Impressions': 'DM',
    'Email_Impressions': 'Email',
    'FB_Impressions': 'FB',
    'Programatic_Impressions': 'Programatic',
    'FTF_Impressions': 'FTF'
}
correct_media_spend_to_channel = {
    'GoogleAds_Spend': 'GoogleAds',
    'Bing_Spend': 'Bing',
    'DM_Spend': 'DM',
    'Email_Spend': 'Email',
    'FB_Spend': 'FB',
    'Programatic_Spend': 'Programatic',
    'FTF_Spend': 'FTF'
}

In [21]:
df.to_csv('mmm_data.csv')

In [22]:
loader = load.CsvDataLoader(
    csv_path="mmm_data.csv",
    kpi_type='revenue',
    coord_to_columns=coord_to_columns,
    media_to_channel=correct_media_to_channel,
    media_spend_to_channel=correct_media_spend_to_channel,
)
data = loader.load()

/usr/local/lib/python3.11/dist-packages/meridian/data/load.py:206: UserWarning: The `population` argument is ignored in a nationally aggregated model. It will be reset to [1]
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/meridian/data/input_data.py:369: UserWarning: Revenue from the `kpi` data is used when `kpi_type`=`revenue`. `revenue_per_kpi` is ignored.
  warnings.warn(


In [ ]:
model_spec = spec.ModelSpec(
    prior=prior_distribution.PriorDistribution(),
    media_effects_dist='log_normal',
    hill_before_adstock=False,
    max_lag=12,
    unique_sigma_for_each_geo=False,
    paid_media_prior_type='roi',
    roi_calibration_period=None,
    rf_roi_calibration_period=None,
    knots=None,
    baseline_geo=None,
    holdout_id=None,
    control_population_scaling_id=None,
)

In [23]:
roi_mu = 0.2
roi_sigma = 0.9
prior = prior_distribution.PriorDistribution(
    roi_m=tfp.distributions.LogNormal(
        roi_mu, roi_sigma, name=constants.ROI_M),
    roi_rf=tfp.distributions.LogNormal(
        roi_mu, roi_sigma, name=constants.ROI_RF
    ),
)
model_spec = spec.ModelSpec(prior=prior)

In [24]:
mmm2 = model.Meridian(input_data=data, model_spec=model_spec)

/usr/local/lib/python3.11/dist-packages/meridian/model/model.py:67: UserWarning: In a nationally aggregated model, the `media_effects_dist` will be reset to `normal`.
  warnings.warn(


In [25]:
%%time
mmm2.sample_prior(500)
mmm2.sample_posterior(n_chains=7, n_adapt=500, n_burnin=500, n_keep=1000)

/usr/local/lib/python3.11/dist-packages/meridian/model/prior_distribution.py:846: UserWarning: Hierarchical distribution parameters must be deterministically zero for national models. tau_g_excl_baseline has been automatically set to Deterministic(0).
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/meridian/model/prior_distribution.py:846: UserWarning: Hierarchical distribution parameters must be deterministically zero for national models. eta_m has been automatically set to Deterministic(0).
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/meridian/model/prior_distribution.py:846: UserWarning: Hierarchical distribution parameters must be deterministically zero for national models. eta_rf has been automatically set to Deterministic(0).
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/meridian/model/prior_distribution.py:846: UserWarning: Hierarchical distribution parameters must be deterministically zero for national models. eta_om has been automatically set to

CPU times: user 3min 9s, sys: 19.5 s, total: 3min 28s
Wall time: 3min 33s


/usr/local/lib/python3.11/dist-packages/arviz/data/inference_data.py:157: UserWarning: trace group is not defined in the InferenceData scheme
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/arviz/data/inference_data.py:1655: UserWarning: trace group is not defined in the InferenceData scheme
  warnings.warn(


In [26]:
model_diagnostics = visualizer.ModelDiagnostics(mmm2)
model_diagnostics.plot_rhat_boxplot()

alt.LayerChart(...)

In [27]:
model_fit = visualizer.ModelFit(mmm2)
model_fit.plot_model_fit()

/usr/local/lib/python3.11/dist-packages/meridian/analysis/analyzer.py:182: UserWarning: The `aggregate_geos` argument is ignored in the national model. It will be reset to `True`.
  warnings.warn(


alt.LayerChart(...)

In [28]:
mmm_summarizer = summarizer.Summarizer(mmm2)

In [29]:
mmm_summarizer.output_model_results_summary('summary_output2.html', filepath=path_)

/usr/local/lib/python3.11/dist-packages/meridian/analysis/analyzer.py:182: UserWarning: The `aggregate_geos` argument is ignored in the national model. It will be reset to `True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/meridian/analysis/analyzer.py:182: UserWarning: The `aggregate_geos` argument is ignored in the national model. It will be reset to `True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/meridian/analysis/analyzer.py:533: UserWarning: Setting `use_kpi=True` has no effect when `kpi_type=REVENUE` since in this case, KPI is equal to revenue.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/tensorflow/python/autograph/impl/api.py:371: UserWarning: Setting `use_kpi=True` has no effect when `kpi_type=REVENUE` since in this case, KPI is equal to revenue.
  return py_builtins.overload_of(f)(*args)


In [30]:
IPython.display.HTML(filename=path_+'summary_output2.html')

Dataset,R-squared,MAPE,wMAPE
All Data,0.15,48%,60%


In [31]:
%%time
budget_optimizer = optimizer.BudgetOptimizer(mmm2)
optimization_results = budget_optimizer.optimize()

CPU times: user 3min 46s, sys: 2.77 s, total: 3min 49s
Wall time: 2min 45s


In [32]:
filepath = path_
optimization_results.output_optimization_summary('optimization_output.html', filepath)

In [33]:
IPython.display.HTML(filename=path_+'optimization_output.html')

Channel,Non-optimized spend,Optimized spend
DM,77%,76%
FTF,10%,11%
GoogleAds,5%,5%
Bing,4%,3%
Email,2%,3%
Programatic,2%,1%
FB,1%,1%


In [34]:
budget_optimizer = optimizer.BudgetOptimizer(mmm2)
optimization_results = budget_optimizer.optimize(
      fixed_budget=False,
      spend_constraint_lower=0.5,
      spend_constraint_upper=0.5,
      target_roi = 1,

)

In [35]:
filepath = path_
optimization_results.output_optimization_summary('optimization_output3.html', filepath)

In [36]:
IPython.display.HTML(filename=path_+'optimization_output3.html')

Channel,Non-optimized spend,Optimized spend
DM,77%,77%
FTF,10%,10%
GoogleAds,5%,5%
Bing,4%,4%
Email,2%,2%
Programatic,2%,2%
FB,1%,1%


In [37]:
optimization_results.nonoptimized_data.to_dataframe().reset_index()

,channel,metric,spend,pct_of_spend,incremental_outcome,pct_of_contribution,effectiveness,roi,mroi,cpik
0,GoogleAds,mean,117000.0,0.051451,2.167552e+05,0.651724,0.406222,1.852604,0.834902,1.235366
1,GoogleAds,median,117000.0,0.051451,1.424280e+05,0.428243,0.266926,1.217333,0.514403,0.821468
2,GoogleAds,ci_lo,117000.0,0.051451,3.304879e+04,0.099369,0.061937,0.282468,0.096486,0.182010
3,GoogleAds,ci_hi,117000.0,0.051451,6.428219e+05,1.932793,1.204718,5.494204,2.498974,3.540221
4,Bing,mean,81000.0,0.035620,1.491840e+05,0.448556,0.255404,1.841780,0.603699,1.248469
5,Bing,median,81000.0,0.035620,9.912173e+04,0.298032,0.169697,1.223725,0.386535,0.817177
6,Bing,ci_lo,81000.0,0.035620,2.215876e+04,0.066625,0.037936,0.273565,0.080131,0.187087
7,Bing,ci_hi,81000.0,0.035620,4.329546e+05,1.301778,0.741221,5.345118,1.785704,3.655440
8,DM,mean,1740000.0,0.765172,2.461200e+06,7.400175,1.032194,1.414482,0.800091,1.402532
9,DM,median,1740000.0,0.765172,1.792285e+06,5.388917,0.751659,1.030049,0.563283,0.970828
